# Animation

In [1]:
import numpy as np
from particle_2d_env import Particle2dEnvironment
from config import run_config

# Initialize environment
run_config["env"]["episode_length"] = 500
env = Particle2dEnvironment(run_config["env"])


/project_ghent/MARL2/collective_env/lib/python3.11/site-packages/gymnasium/spaces/box.py:130: UserWarning: WARN: Box bound precision lowered by casting to float32
  gym.logger.warn(f"Box bound precision lowered by casting to {self.dtype}")


In [2]:
import ray
from ray.rllib.algorithms.algorithm import Algorithm

def process_observations(observations, current_observation, agent_ids, termination=None):
    loc_x = [current_observation[key][0] if key in current_observation else 0 for key in agent_ids]
    loc_y = [current_observation[key][1] if key in current_observation else 0 for key in agent_ids]
    heading = [current_observation[key][2] if key in current_observation else 0 for key in agent_ids]
    if termination:
        still_in_the_game = [1 if not termination[key] else 0 for key in agent_ids]
    else:
        still_in_the_game = [1 for _ in agent_ids]
    observations["loc_x"].append(np.array(loc_x))
    observations["loc_y"].append(np.array(loc_y))
    observations["heading"].append(np.array(heading))
    observations["still_in_the_game"].append(np.array(still_in_the_game))

    return observations

def collect_observations(path_to_checkpoint, env, number_of_steps=300):
    # Load the checkpoint
    algo = Algorithm.from_checkpoint(path_to_checkpoint)
    
    observations = {"loc_x": [], "loc_y": [], "heading": [], "still_in_the_game": []}
    
    # Run simulation
    observation, _ = env.reset()
    for step_count in range(number_of_steps):
        actions = {
            key: algo.compute_single_action(
                obs_batch, policy_id="prey" if env.agents[key].agent_type == 0 else "predator"
            ) for key, obs_batch in observation.items()
        }
    
        observation, _, termination, _, _ = env.step(actions)
        observations = process_observations(observations, observation, env._agent_ids, termination)
    
    grid_diagonal = env.grid_diagonal
    observations["loc_x"] = np.array(observations["loc_x"]) * grid_diagonal
    observations["loc_y"] = np.array(observations["loc_y"]) * grid_diagonal
    observations["heading"] = np.array(observations["heading"])
    observations["still_in_the_game"] = np.array(observations["still_in_the_game"])
    
    env.close()
    ray.shutdown()
    
    return observations

In [3]:
import os

path_to_checkpoint = os.getcwd() + "/ray_results" + "/PPO_2024-02-27_11-21-58/PPO_Particle2dEnvironment_6bd8e_00000_0_2024-02-27_11-21-58/checkpoint_000015"
observations = collect_observations(path_to_checkpoint, env)

/project_ghent/MARL2/collective_env/lib/python3.11/site-packages/ray/rllib/algorithms/algorithm.py:483: RayDeprecationWarning: This API is deprecated and may be removed in future Ray releases. You could suppress this warning by setting env variable PYTHONWARNINGS="ignore::DeprecationWarning"
`UnifiedLogger` will be removed in Ray 2.7.
  return UnifiedLogger(config, logdir, loggers=None)
/project_ghent/MARL2/collective_env/lib/python3.11/site-packages/ray/tune/logger/unified.py:53: RayDeprecationWarning: This API is deprecated and may be removed in future Ray releases. You could suppress this warning by setting env variable PYTHONWARNINGS="ignore::DeprecationWarning"
The `JsonLogger interface is deprecated in favor of the `ray.tune.json.JsonLoggerCallback` interface and will be removed in Ray 2.7.
  self._loggers.append(cls(self.config, self.logdir, self.trial))
/project_ghent/MARL2/collective_env/lib/python3.11/site-packages/ray/tune/logger/unified.py:53: RayDeprecationWarning: This AP

In [4]:
from IPython.display import HTML
from animation import generate_animation_3d

ani = generate_animation_3d(observations, env, fps=10)

HTML(ani.to_html5_video())

# Metrics

In [ ]:
import os
from analysis import calculate_dos

# List all checkpoint directories
checkpoint_folder = os.getcwd() + "/ray_results" + "/PPO_2024-02-19_21-16-08/PPO_CustomEnvironment_1961a_00000_0_2024-02-19_21-16-08/"
checkpoint_dirs = [d for d in os.listdir(checkpoint_folder) if d.startswith("checkpoint_")]

dos = []
for checkpoint_dir in checkpoint_dirs:
    # Construct the path to the current checkpoint
    path_to_checkpoint = os.path.join(checkpoint_folder, checkpoint_dir)   # or path_to_checkpoint = checkpoint_folder + "checkpoint_" + str(i).zfill(6)
    observations = collect_observations(path_to_checkpoint, env)
    dos.append(calculate_dos(observations, env.stage_size))


In [ ]:
import matplotlib.pyplot as plt

# plot dos curve
plt.plot(dos)
plt.show()

# Action distribution visualisation

In [ ]:
import os
from ray.rllib.algorithms.algorithm import Algorithm

path_to_checkpoint = os.getcwd() + "/ray_results" + "/PPO_2024-02-24_00-21-49/PPO_Particle2dEnvironment_b3919_00000_0_2024-02-24_00-21-49/checkpoint_000033"

algo = Algorithm.from_checkpoint(path_to_checkpoint)

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from ipywidgets import interactive, interact, FloatSlider, VBox, HBox
from IPython.display import display
import torch

# The policy is used to define the functions
policy = algo.get_policy("prey")

# Calculate vision endpoints
left_x = env.max_seeing_distance * np.cos(-env.max_seeing_angle)
left_y = env.max_seeing_distance * np.sin(-env.max_seeing_angle)
right_x = env.max_seeing_distance * np.cos(env.max_seeing_angle)
right_y = env.max_seeing_distance * np.sin(env.max_seeing_angle)

def update_plot_observation(r, theta, heading):
    fig, ax_obs = plt.subplots()

    x = r * np.cos(theta)
    y = r * np.sin(theta)
    
    ax_obs.plot(x, y, 'ro') # Red point at the polar coordinates for the observed fish
    ax_obs.plot(0, 0, 'bo') # Blue dot in the middle for the fish observing
    ax_obs.set_xlim(-env.stage_size, env.stage_size)
    ax_obs.set_ylim(-env.stage_size, env.stage_size)
    ax_obs.axhline(0, color='black', linewidth=0.5)
    ax_obs.axvline(0, color='black', linewidth=0.5)
    ax_obs.set_title("setting")

    # Length of the heading line
    heading_length = 1  # Adjust as needed
    # Calculate the end point of the heading line
    heading_x = x + heading_length * np.cos(heading)
    heading_y = y + heading_length * np.sin(heading)

    # Draw the heading line
    ax_obs.plot([x, heading_x], [y, heading_y], 'g-')  # Green line for heading

    # Draw the field of vision lines
    ax_obs.plot([0, left_x], [0, left_y], 'b--')  # Left vision line
    ax_obs.plot([0, right_x], [0, right_y], 'b--')  # Right vision line

# Observations that won't move
observation = np.zeros(env.observation_size)
observation[0:5] = [10 / env.grid_diagonal, 10 / env.grid_diagonal, 0, 0.1 / env.max_speed_prey, 0.1 / env.max_speed_prey]
for j in [0, 1, 2, 3, 4, 5]:
    base_index = 5  + j * env.num_observed_properties
    if env.use_speed_observation:
        observation[base_index + 3] = 0                   # speed 1
        observation[base_index + 4] = 0                   # speed 2
        observation[base_index + 5] = 0                   # prey/predator type
    else:
        observation[base_index + 3] = 0                   # prey/predator type

#  Calculate the x values for each bell curve
x_acc = np.linspace(-0.1, env.max_acceleration_prey, 100)
x_turn = np.linspace(-env.max_turn, env.max_turn, 100)

# Function to update the plot
def update_plot_actions(r, theta, heading):
    # Create a figure with 2 subplots (one above the other)
    fig, ax_actions = plt.subplots(2, 1, figsize=(8, 6))
    
    for j in [0, 1, 2, 3, 4, 5]:
        base_index = 5  + j * env.num_observed_properties
        observation[base_index] =    r / env.grid_diagonal   # relative normalized position x
        observation[base_index + 1] = theta / (2 * np.pi)    # relative normalized position y
        observation[base_index + 2] = heading / (2 * np.pi)  # relative heading (always the same here)

    mean_acc, mean_turn, log_std_acc, log_std_turn = policy.compute_single_action(observation)[2]["action_dist_inputs"].tolist()
    
    std_acc = np.exp(log_std_acc)
    std_turn = np.exp(log_std_turn)
    
    # Calculate the y values for each bell curve
    y_acc = (1 / (np.sqrt(2 * np.pi) * std_acc)) * np.exp(-0.5 * ((x_acc - mean_acc) / std_acc) ** 2)
    y_turn = (1 / (np.sqrt(2 * np.pi) * std_turn)) * np.exp(-0.5 * ((x_turn - mean_turn) / std_turn) ** 2)

 

    # Plotting on the acceleration amplitude subplot
    ax_actions[0].plot(x_acc, y_acc, color='blue', linestyle='-', linewidth=2, label='Amplitude')
    ax_actions[0].grid(True)
    ax_actions[0].legend()
    ax_actions[0].set_title("action amplitude")
    
    # Plotting on the turn angle subplot
    ax_actions[1].plot(x_turn, y_turn, color='red', linestyle='--', linewidth=2, label='Orientation')
    ax_actions[1].grid(True)
    ax_actions[1].legend()
    ax_actions[1].set_title("action turn")
    
    # Adjust layout to not overlap plots
    plt.tight_layout()
    plt.show()

# Interactive sliders for radius and angle
radius_slider = FloatSlider(min=0, max=env.max_seeing_distance, step=0.1, value=20, description='Distance (r)', orientation='horizontal')
theta_slider = FloatSlider(min=-np.pi, max=np.pi, step=0.1, value= 0.5, description='Angle (θ)', orientation='horizontal')
heading_slider = FloatSlider(min=-np.pi, max=np.pi, step=0.1, value= 0.5, description='Heading (θ)', orientation='horizontal')


# Create the interactive plot
interactive_plot1 = interactive(update_plot_observation, r=radius_slider, theta=theta_slider, heading=heading_slider)
interactive_plot2 = interactive(update_plot_actions, r=radius_slider, theta=theta_slider, heading=heading_slider)

# Extract the output and set the height
output1 = interactive_plot1.children[-1]
output2 = interactive_plot2.children[-1]

# Adjust the output layout to have the sliders at the bottom of the first plot
slider_box = VBox([radius_slider, theta_slider, heading_slider])
left_box = VBox([output1, slider_box])
full_plot = HBox([left_box, output2])
display(full_plot)

# Atraction, aligment and collision zones 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches


def compute_zones(r, theta):
    # Define a range of heading values to simulate, e.g., [0, 2*pi] in steps
    heading_values = np.linspace(-np.pi, np.pi, num=10)  # Adjust num for finer granularity
    alignment_info = []

    for heading in heading_values:
        observation = np.zeros(env.observation_size)
        # Example initialization, adjust as needed
        observation[0:5] = [10 / env.grid_diagonal, 10 / env.grid_diagonal, 0, 0.1 / env.max_speed_prey, 0.1 / env.max_speed_prey]

        # Set up observation based on the current simulated heading
        for j in range(6):  # Assuming 6 observations for simplification
            base_index = 5 + j * env.num_observed_properties
            observation[base_index] = r / env.grid_diagonal
            observation[base_index + 1] = theta / (2 * np.pi)
            observation[base_index + 2] = heading / (2 * np.pi)
            if env.use_speed_observation:
                observation[base_index + 3:base_index + 6] = [0, 0, 0]
            else:
                observation[base_index + 3] = 0

        action_dist_inputs = policy.compute_single_action(observation)[2]["action_dist_inputs"].tolist()
        mean_acc, mean_turn, log_std_acc, log_std_turn = action_dist_inputs
        std_acc = np.exp(log_std_acc)
        std_turn = np.exp(log_std_turn)

        aligment_turn = (mean_turn - heading) % (2 * np.pi) - np.pi
        is_aligned = (-env.max_turn <= mean_turn <= env.max_turn) and np.sign(mean_turn) == np.sign(aligment_turn)

        # Capture alignment information
        alignment_info.append(is_aligned)

    alignment_ratio = sum(alignment_info) / len(alignment_info)
    
    # Define thresholds
    collision_threshold = std_turn * 2
    attraction_threshold = std_turn * 0.5
    alignment_threshold = 0.6

    # Zone computation
    if alignment_ratio >= alignment_threshold:
        return 'alignment', alignment_ratio 
    elif np.abs(mean_turn - theta) <= attraction_threshold:
        zone = 'attraction'
    elif np.abs(mean_turn - theta) > collision_threshold:
        zone = 'collision'
    else:
        zone = 'indeterminate'  # Conditions for other zones not met

    return zone


def plot_zones():
    plt.figure(figsize=(10, 10))

    # Assuming env.max_seeing_distance, env.stage_size, and other parameters are defined
    for r in np.linspace(0, env.max_seeing_distance, 50):
        for theta in np.linspace(-env.max_seeing_angle, env.max_seeing_angle, 100):
            zone = compute_zones(r, theta)
            x = r * np.cos(theta)
            y = r * np.sin(theta)

            if zone == 'collision':
                color = 'red'
            elif zone == 'attraction':
                color = 'green'
            elif zone == 'indeterminate':
                color = 'grey'
            else:  # alignment
                color = 'blue'

            plt.plot(x, y, marker='.', color=color)

    # Plot formatting, assuming you've defined these variables
    plt.xlim(-env.stage_size, env.stage_size)
    plt.ylim(-env.stage_size, env.stage_size)
    plt.axhline(0, color='black', linewidth=0.5)
    plt.axvline(0, color='black', linewidth=0.5)

    # Add legend
    collision_patch = patches.Patch(color='red', label='Collision Zone')
    attraction_patch = patches.Patch(color='green', label='Attraction Zone')
    alignment_patch = patches.Patch(color='blue', label='Alignment Zone')
    plt.legend(handles=[collision_patch, attraction_patch, alignment_patch])

    plt.show()

# Call the plotting function
plot_zones()


# Brouillon

In [ ]:
policy = algo.get_policy("prey")

policy.make_model_and_action_dist

In [ ]:
import torch

fc_net_output = policy.model.encoder.actor_encoder.net.mlp(torch.Tensor(toy_vector))

outputs = policy.model.pi.net(fc_net_output).tolist()

In [ ]:
policy.model.config

In [ ]:
model_config = policy.model.config
catalog = model_config.catalog_class(model_config.observation_space, model_config.action_space, model_config.model_config_dict)


In [ ]:
action_dist_class = catalog.get_action_dist_cls(framework="torch")

fc_net_output = policy.model.encoder.actor_encoder.net.mlp(torch.Tensor(toy_vector))

action_dist_inputs = policy.model.pi.net(fc_net_output)

action_dist = action_dist_class.from_logits(action_dist_inputs)
actions = action_dist.sample().numpy()
actions

In [ ]:
action_dist

In [ ]:
action_dist_class

In [ ]:
policy.model

# Network visualization

In [ ]:
# Example shape
policy_id = "prey"
print(algo.get_policy(policy_id).get_weights()['pi.net.mlp.0.bias'].shape)
# We create a subdictionnary with the interresting layers
actor_weights = {}
for key, value in algo.get_policy(policy_id).get_weights().items():
    if "critic_encoder" not in key and "vf." not in key:
        actor_weights[key] = value

actor_weights.keys()

In [ ]:
import numpy as np

from graph_tool.all import *

def create_graph(neural_network):
    g = Graph(directed=True)
    
    # Create property maps for vertex and edge labels and edge width
    v_label = g.new_vertex_property("string")
    e_width = g.new_edge_property("double")
    pos = g.new_vertex_property("vector<double>")
    
    max_neurons = max(len(neural_network[key]) for key in neural_network if 'weight' in key)

    def add_layer_and_set_positions(neurons, x_pos, pos):
        layer_vertices = [g.add_vertex() for _ in neurons]
        starting_y = (max_neurons - len(layer_vertices)) / 2
        for i, v in enumerate(layer_vertices):
            pos[v] = (x_pos, starting_y + len(layer_vertices) - 1 - i)
        return layer_vertices
    
    ## VERTEX ##
    # Input Layer
    layers = [add_layer_and_set_positions(neural_network['encoder.actor_encoder.net.mlp.0.weight'].T, 0, pos)]
    
    # Hiden Layers
    x_gap = 20 # gap between layers
    biases_keys = [key for key in neural_network if ".bias" in key and "actor_encoder" in key]
    for i, biases_key in enumerate(biases_keys):
        # Add vertices for the current layer and set their positions
        layers.append(add_layer_and_set_positions(neural_network[biases_key], x_gap*(i+1), pos))
        
    # Output Layer
    output_neurons = add_layer_and_set_positions(neural_network['pi.net.mlp.0.bias'], x_gap*4, pos)

    ## EDGES ##
    # Set labels and add edges for input-hidden and hidden-hidden layer
    weights_keys = [key for key in neural_network if ".weight" in key and "actor_encoder" in key]
    for k, weights_key in enumerate(weights_keys):
        for i, hidden_neuron in enumerate(layers[k]):
            for j, next_hidden_neuron in enumerate(layers[k+1]):
                e = g.add_edge(hidden_neuron, next_hidden_neuron)
                weight = neural_network[weights_key].T[i][j]
                e_width[e] = weight

    # Set labels and add edges for hidden-output layer
    for j, output_neuron in enumerate(output_neurons):
        for i, hidden_neuron in enumerate(layers[-1]):
            e = g.add_edge(hidden_neuron, output_neuron)
            weight = neural_network['pi.net.mlp.0.weight'].T[i][j]
            e_width[e] = weight

    
    # LABELS
    # Set neuron labels (optional, for clarity)
    for v in layers[0]:
        v_label[v] = "I"
    for k, hidden_neurons in enumerate(layers):
        for v in hidden_neurons:
            v_label[v] = "H"
    for v in output_neurons:
        v_label[v] = "O"

    return g, pos, v_label, e_width

# Example usage with the same nn_wandb
g, pos, v_label, e_width = create_graph(actor_weights)

# Draw the graph
graph_draw(g, pos=pos, vertex_text=v_label, edge_text=None, edge_pen_width=e_width, vertex_size=15, vertex_font_size=10, edge_font_size=10, output_size=(800, 800))
    

## Can we reproduce the action by doing a forward pass on the network ?

In [ ]:
import numpy as np

def neural_net_activation(input_vector, neural_dic):
    # Ensuring the input is a numpy array
    input_vector = np.array(input_vector)

    # Layer activations
    activations = {}

    # First three hidden layers
    for i in range(3):
        weight_key = f"encoder.actor_encoder.net.mlp.{2*i}.weight"
        bias_key = f"encoder.actor_encoder.net.mlp.{2*i}.bias"

        if i == 0:
            layer_input = input_vector
        else:
            layer_input = activations[f"layer_{i}"]

        # Calculate the layer output
        z = np.dot(layer_input, neural_dic[weight_key].T) + neural_dic[bias_key]
        activations[f"layer_{i+1}"] = np.tanh(z)

    # Output layer
    weight_key = "pi.net.mlp.0.weight"
    bias_key = "pi.net.mlp.0.bias"
    output_layer_input = activations["layer_3"]
    z = np.dot(output_layer_input, neural_dic[weight_key].T) + neural_dic[bias_key]
    activations["output"] = np.tanh(z)

    return activations

toy_vector = [0.95600354, 0.36926809,
 0.07873756, 0.38905384, 0.92927526, 0.08713003, 0.61284082,
 0.82746801, 0.36660529, 0.89503505, 0.04707359, 0.97744959,
 0.28597701, 0.96454964, 0.29067754, 0.74090134, 0.8245886, 0.96454964, 0.29067754, 0.74090134, 0.8245886,
             0.96454964, 0.29067754, 0.74090134, 0.8245886, 0.96454964, 0.29067754, 0.29067754, 0.29067754]

activations = neural_net_activation(toy_vector, actor_weights)

activations["output"]